In [1]:
# Dependencies and Setup
import pandas as pd
import datetime as dt

# File to Load (Remember to Change These)
production = "resources/production.csv"
site = "resources/site_data.csv"
weather = "resources/weather_data.csv"

In [2]:
# Read production and site data and store into Pandas Data Frames
production_data = pd.read_csv(production)
site_data = pd.read_csv(site)
site_data = site_data[["id", "azimuth", "tilt", "capacity(kWp)", "lat", "lon"]]
site_data.head()

,id,azimuth,tilt,capacity(kWp),lat,lon
0,1,180,15,10.20,19.354622,-99.165995
1,2,128,12,5.72,19.395768,-99.270309
2,3,155,15,6.95,19.415420,-99.213157
3,6,152,15,3.12,18.955723,-99.252346
4,7,106,9,3.12,19.386032,-99.263238


In [3]:
# Get production data and change units to kWh
production_data["production(kWh)"] = production_data["produccion(Wh)"] / 1000
production_data = production_data.drop("produccion(Wh)", axis=1)
production_data.head()

,id,fecha,production(kWh)
0,1,27/05/2020,34.047254
1,1,28/05/2020,37.065480
2,1,29/05/2020,37.592620
3,1,30/05/2020,34.483457
4,1,31/05/2020,33.053957


In [4]:
# Get specific production indicator
production_data = production_data.merge(site_data, how='inner', on='id')
production_data['specific_prod(kWh/kWp)'] = production_data['production(kWh)']/production_data['capacity(kWp)']
production_data = production_data.rename(columns = {'fecha': 'date'})
production_data = production_data[["id", "date", "production(kWh)", "specific_prod(kWh/kWp)"]]
production_data.head()

,id,date,production(kWh),specific_prod(kWh/kWp)
0,1,27/05/2020,34.047254,3.337966
1,1,28/05/2020,37.065480,3.633871
2,1,29/05/2020,37.592620,3.685551
3,1,30/05/2020,34.483457,3.380731
4,1,31/05/2020,33.053957,3.240584


In [5]:
# Change date format
production_data['date'] = pd.to_datetime(production_data['date'], format="%d/%m/%Y")

In [6]:
# Get weather data
weather_data = pd.read_csv(weather)

In [7]:
# Get columns from weather data
weather_data.columns

Index(['date', 'id', 'uvIndex', 'sunHour', 'cloudcover', 'maxtempC',
       'mintempC', 'precipitation', 'sunrise', 'sunset'],
      dtype='object')

In [8]:
# Print weather data head to explore the columns and formatting
weather_data.head()

,date,id,uvIndex,sunHour,cloudcover,maxtempC,mintempC,precipitation,sunrise,sunset
0,2017-01-01,1,4.0,11.0,7.0,20.0,8.0,0.0,08:11 AM,07:10 PM
1,2017-01-02,1,4.0,11.0,2.0,21.0,8.0,0.0,08:11 AM,07:11 PM
2,2017-01-03,1,4.0,11.0,2.0,22.0,9.0,0.0,08:11 AM,07:12 PM
3,2017-01-04,1,4.0,11.0,2.0,23.0,9.0,0.0,08:11 AM,07:12 PM
4,2017-01-05,1,4.0,11.0,0.0,22.0,7.0,0.0,08:12 AM,07:13 PM


In [9]:
# Change date data type
weather_data["date"] = pd.to_datetime(weather_data['date'], yearfirst=True, infer_datetime_format=True)
weather_data.head()

,date,id,uvIndex,sunHour,cloudcover,maxtempC,mintempC,precipitation,sunrise,sunset
0,2017-01-01,1,4.0,11.0,7.0,20.0,8.0,0.0,08:11 AM,07:10 PM
1,2017-01-02,1,4.0,11.0,2.0,21.0,8.0,0.0,08:11 AM,07:11 PM
2,2017-01-03,1,4.0,11.0,2.0,22.0,9.0,0.0,08:11 AM,07:12 PM
3,2017-01-04,1,4.0,11.0,2.0,23.0,9.0,0.0,08:11 AM,07:12 PM
4,2017-01-05,1,4.0,11.0,0.0,22.0,7.0,0.0,08:12 AM,07:13 PM


In [10]:
# Merge weather and production data into a single Data Frame
daily_data = pd.merge(production_data, weather_data, how='right', on=['id', 'date'])
daily_data.head()

,id,date,production(kWh),specific_prod(kWh/kWp),uvIndex,sunHour,cloudcover,maxtempC,mintempC,precipitation,sunrise,sunset
0,1,2020-05-27,34.047254,3.337966,6.0,11.6,17.0,28.0,14.0,0.7,06:58 AM,08:09 PM
1,1,2020-05-28,37.065480,3.633871,5.0,10.4,35.0,26.0,13.0,8.9,06:58 AM,08:10 PM
2,1,2020-05-29,37.592620,3.685551,4.0,8.0,51.0,23.0,12.0,17.7,06:58 AM,08:10 PM
3,1,2020-05-30,34.483457,3.380731,4.0,11.6,36.0,24.0,10.0,0.8,06:58 AM,08:11 PM
4,1,2020-05-31,33.053957,3.240584,4.0,9.2,48.0,23.0,10.0,6.3,06:58 AM,08:11 PM


In [11]:
# Drop nulls
daily_data = daily_data.dropna()

In [12]:
# Export to csv files
daily_data.to_csv("resources/db_files/daily_data.csv", index=False)
site_data.to_csv("resources/db_files/site_data.csv", index=False)